In [1]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper



In [2]:
## now over here we will be making out a wrapper that can make sure that we can first set up the wrapper around the wikipida with the content taht we want
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
##making the tool setting it up with this wrapper to make sure we use it to interact with the wikipedia
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [3]:
##this one is the custom tool for this requirement for the webbase loeadr
##the above ones are used for the already existing apis
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader= WebBaseLoader("https://docs.smith.langchain.com/")
docs= loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
db = Chroma.from_documents(documents, embeddings)
retriever=db.as_retriever()
retriever





USER_AGENT environment variable not set, consider setting it to identify your requests.
C:\Users\ppriy\AppData\Local\Temp\ipykernel_10348\3731178355.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


VectorStoreRetriever(tags=['Chroma', 'HuggingFaceEmbeddings'], vectorstore=<langchain_community.vectorstores.chroma.Chroma object at 0x0000029514B00AD0>, search_kwargs={})

In [4]:
##making a tools that can retrive the info on the langsmith
from langchain.tools.retriever import create_retriever_tool

retriever_tool=create_retriever_tool(retriever,"langsmith_search","Search for info about langsmith. for any question about langsmith u must use this tool")

In [6]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

Arxiv_wrapper=ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv= ArxivQueryRun(api_wrapper=Arxiv_wrapper)
arxiv.name

'arxiv'

In [7]:
##now combining / chaining the above made tools
tools=[wiki,arxiv,retriever_tool]

In [8]:
##now to use the agents in order to perform the search on this combined tools thing lets make a new llm
from dotenv import load_dotenv

load_dotenv()
import os

os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")
from langchain_google_genai import ChatGoogleGenerativeAI 

llm = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

In [9]:
##u can make your own prompt using promttemplate
##but over here we will make a new prompt and get that from hub
from langchain import hub
prompt= hub.pull("hwchase17/openai-functions-agent")

prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='You are a helpful assistant'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='{input}'), additional_kwargs={}),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [10]:
#now lets create the agent in order to perform the retrieval
from langchain.agents import create_openai_functions_agent
agent=create_openai_functions_agent(llm,tools,prompt)

In [11]:
##to execute the agent operation we use agent executer
from langchain.agents import AgentExecutor
agent_executer=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [12]:
agent_executer.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...


Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 2.0 seconds as it raised NotFound: 404 Publisher Model `projects/generativelanguage-ga/locations/europe-west4/publishers/google/models/gemini-1.5-flash-002` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_retry in 4.0 seconds as it raised NotFound: 404 Publisher Model `projects/generativelanguage-ga/locations/europe-west4/publishers/google/models/gemini-1.5-flash-002` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions.
Retrying langchain_google_genai.chat_models._chat_with_retry.<locals>._chat_with_r

NotFound: 404 Publisher Model `projects/generativelanguage-ga/locations/europe-west4/publishers/google/models/gemini-1.5-flash-002` was not found or your project does not have access to it. Please ensure you are using a valid model version. For more information, see: https://cloud.google.com/vertex-ai/generative-ai/docs/learn/model-versions